In [1]:
def my_decorator(some_function):

    def wrapper():

        num = 10

        if num == 10:
            print("Yes!")
        else:
            print("No!")

        some_function()

        print("Something is happening after some_function() is called.")

    return wrapper


In [3]:
from energy_py import make_paths, make_logger, EternityVisualizer
from energy_py.envs import BatteryEnv
from energy_py.agents import RandomAgent

In [4]:
def experiment():
    """
    A decorator used to wrap experiments
    """
    def outer_wrapper(my_expt_func):

        def wrapper(**kwargs):
            LOG_STATUS = False
            EPISODE_LENGTH = 10
            EPISODES = 20
            EPISODE_RANDOM = False
            OUTPUT_RESULTS = 5
            DISCOUNT=0.9

            data_path = kwargs['data_path']
            base_path = kwargs['base_path']
            env = kwargs['env']
            agent = kwargs['agent']

            print('before agent expt')
            paths = make_paths(base_path)
            BRAIN_PATH = paths['brain']
            RESULTS_PATH = paths['results']
            ARGS_PATH = paths['args']
            LOG_PATH = paths['logs']

            logger = make_logger(LOG_PATH, LOG_STATUS)

            env = env(data_path, 
                      episode_length=EPISODE_LENGTH,
                      episode_random=EPISODE_RANDOM)
            
            agent_outputs, env_outputs = my_expt_func(env, agent, 
                                                      EPISODES, DISCOUNT, 
                                                      OUTPUT_RESULTS, RESULTS_PATH)

            return agent_outputs, env_outputs
        
        return wrapper
    
    return outer_wrapper

@experiment()
def no_learn_expt(env, agent, EPISODES, DISCOUNT, OUTPUT_RESULTS, RESULTS_PATH):
    
    agent = agent(env, DISCOUNT)

    for episode in range(1, EPISODES):

        #  initialize before starting episode
        done, step = False, 0
        observation = env.reset(episode)

        #  while loop runs through a single episode
        while done is False:
            #  we cheat a little bit here by grabbing the observation as a
            #  pandas series
            #  just want to pull out the timestamp
            time = env.observation_ts.index[env.steps]
            #  select an action
            action = agent.act(observation=observation, timestamp=time)
            #  take one step through the environment
            next_observation, reward, done, info = env.step(action)
            #  store the experience
            agent.memory.add_experience(observation, action, reward,
                                        next_observation, done, step, episode)
            step += 1
            observation = next_observation

        if episode % OUTPUT_RESULTS == 0:
            #  collect data from the agent & environment
            hist = EternityVisualizer(agent,
                                      env,
                                      results_path=RESULTS_PATH)

            agent_outputs, env_outputs = hist.output_results(save_data=True)
            
    return agent_outputs, env_outputs

            

In [5]:
agent_outputs, env_outputs = no_learn_expt(data_path='', base_path='base', env=BatteryEnv, agent=RandomAgent)

2017-12-20 12:58:33,298 [DEBUG]energy_py.envs.env_core: Reset environment
2017-12-20 12:58:33,302 [INFO]energy_py.envs.env_ts: Ep none starting at 2017-01-01 00:00:00
2017-12-20 12:58:33,304 [DEBUG]energy_py.envs.battery.battery_env: initial state is [[ 40.   0.]]
2017-12-20 12:58:33,305 [DEBUG]energy_py.envs.battery.battery_env: initial obs is [[ 40.  40.  40.  40.  40.  40.  40.  40.  40.  40.   0.]]
2017-12-20 12:58:33,306 [DEBUG]energy_py.envs.battery.battery_env: initial charge is 0.0
2017-12-20 12:58:33,307 [DEBUG]energy_py.envs.env_core: Reset environment
2017-12-20 12:58:33,309 [INFO]energy_py.envs.env_ts: Ep 1 starting at 2017-01-01 00:00:00
2017-12-20 12:58:33,315 [DEBUG]energy_py.envs.battery.battery_env: initial state is [[ 40.   0.]]
2017-12-20 12:58:33,317 [DEBUG]energy_py.envs.battery.battery_env: initial obs is [[ 40.  40.  40.  40.  40.  40.  40.  40.  40.  40.   0.]]
2017-12-20 12:58:33,318 [DEBUG]energy_py.envs.battery.battery_env: initial charge is 0.0
2017-12-20 12

2017-12-20 12:58:33,383 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 1.076 MW
2017-12-20 12:58:33,384 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.009 MWh
2017-12-20 12:58:33,384 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.968 MW
2017-12-20 12:58:33,385 [DEBUG]energy_py.envs.battery.battery_env: reward is -3.586 $/5min
2017-12-20 12:58:33,386 [DEBUG]energy_py.agents.memory: adding exp episode 1 step 7
2017-12-20 12:58:33,386 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-20 12:58:33,387 [DEBUG]energy_py.envs.env_core: Episode 1 - Step 8
2017-12-20 12:58:33,388 [DEBUG]energy_py.envs.battery.battery_env: step is 8.000
2017-12-20 12:58:33,388 [DEBUG]energy_py.envs.battery.battery_env: action was [ 1.01377244  0.98854545]
2017-12-20 12:58:33,389 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.108 MWh
2017-12-20 12:58:33,389 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.110 MWh
2017-12-20 12:58:33,390 [DEBUG]energy_py.env

2017-12-20 12:58:33,452 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.465 MW
2017-12-20 12:58:33,453 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.004 MWh
2017-12-20 12:58:33,453 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.419 MW
2017-12-20 12:58:33,454 [DEBUG]energy_py.envs.battery.battery_env: reward is -1.551 $/5min
2017-12-20 12:58:33,455 [DEBUG]energy_py.agents.memory: adding exp episode 2 step 6
2017-12-20 12:58:33,456 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-20 12:58:33,456 [DEBUG]energy_py.envs.env_core: Episode 2 - Step 7
2017-12-20 12:58:33,457 [DEBUG]energy_py.envs.battery.battery_env: step is 7.000
2017-12-20 12:58:33,457 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.0946192   0.01307201]
2017-12-20 12:58:33,458 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.118 MWh
2017-12-20 12:58:33,459 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.124 MWh
2017-12-20 12:58:33,459 [DEBUG]energy_py.env

before agent expt


2017-12-20 12:58:33,476 [DEBUG]energy_py.envs.battery.battery_env: initial obs is [[ 40.  40.  40.  40.  40.  40.  40.  40.  40.  40.   0.]]
2017-12-20 12:58:33,476 [DEBUG]energy_py.envs.battery.battery_env: initial charge is 0.0
2017-12-20 12:58:33,477 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-20 12:58:33,478 [DEBUG]energy_py.envs.env_core: Episode 3 - Step 0
2017-12-20 12:58:33,479 [DEBUG]energy_py.envs.battery.battery_env: step is 0.000
2017-12-20 12:58:33,480 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.04283991  0.81384357]
2017-12-20 12:58:33,481 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.000 MWh
2017-12-20 12:58:33,482 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.000 MWh
2017-12-20 12:58:33,482 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.000 MW
2017-12-20 12:58:33,483 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.000 MWh
2017-12-20 12:58:33,484 [DEBUG]energy_py.envs.battery.battery_env: net rate i

...

2017-12-20 12:58:48,352 [DEBUG]energy_py.envs.env_core: Reset environment
2017-12-20 12:58:48,355 [INFO]energy_py.envs.env_ts: Ep 16 starting at 2017-01-01 00:00:00
2017-12-20 12:58:48,356 [DEBUG]energy_py.envs.battery.battery_env: initial state is [[ 40.   0.]]
2017-12-20 12:58:48,357 [DEBUG]energy_py.envs.battery.battery_env: initial obs is [[ 40.  40.  40.  40.  40.  40.  40.  40.  40.  40.   0.]]
2017-12-20 12:58:48,358 [DEBUG]energy_py.envs.battery.battery_env: initial charge is 0.0
2017-12-20 12:58:48,358 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-20 12:58:48,359 [DEBUG]energy_py.envs.env_core: Episode 16 - Step 0
2017-12-20 12:58:48,360 [DEBUG]energy_py.envs.battery.battery_env: step is 0.000
2017-12-20 12:58:48,360 [DEBUG]energy_py.envs.battery.battery_env: action was [ 1.92521245  1.01956133]
2017-12-20 12:58:48,361 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.000 MWh
2017-12-20 12:58:48,361 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.

2017-12-20 12:58:48,453 [DEBUG]energy_py.envs.env_core: Episode 16 - Step 8
2017-12-20 12:58:48,455 [DEBUG]energy_py.envs.battery.battery_env: step is 8.000
2017-12-20 12:58:48,456 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.48626176  0.97825623]
2017-12-20 12:58:48,457 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.122 MWh
2017-12-20 12:58:48,457 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.081 MWh
2017-12-20 12:58:48,458 [DEBUG]energy_py.envs.battery.battery_env: gross rate is -0.492 MW
2017-12-20 12:58:48,458 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.000 MWh
2017-12-20 12:58:48,459 [DEBUG]energy_py.envs.battery.battery_env: net rate is -0.492 MW
2017-12-20 12:58:48,459 [DEBUG]energy_py.envs.battery.battery_env: reward is 1.640 $/5min
2017-12-20 12:58:48,460 [INFO]energy_py.envs.battery.battery_env: Episode 16 - Total reward -6.10
2017-12-20 12:58:48,461 [DEBUG]energy_py.agents.memory: adding exp episode 16 step 8
2017-12-20 12:

2017-12-20 12:58:48,551 [DEBUG]energy_py.envs.env_core: Episode 17 - Step 7
2017-12-20 12:58:48,551 [DEBUG]energy_py.envs.battery.battery_env: step is 7.000
2017-12-20 12:58:48,552 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.18129026  0.70369765]
2017-12-20 12:58:48,553 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.164 MWh
2017-12-20 12:58:48,553 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.120 MWh
2017-12-20 12:58:48,554 [DEBUG]energy_py.envs.battery.battery_env: gross rate is -0.522 MW
2017-12-20 12:58:48,554 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.000 MWh
2017-12-20 12:58:48,555 [DEBUG]energy_py.envs.battery.battery_env: net rate is -0.522 MW
2017-12-20 12:58:48,556 [DEBUG]energy_py.envs.battery.battery_env: reward is 1.741 $/5min
2017-12-20 12:58:48,557 [DEBUG]energy_py.agents.memory: adding exp episode 17 step 7
2017-12-20 12:58:48,558 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-20 12:58:48,559 [DEBUG]energy_py.

2017-12-20 12:58:48,649 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-20 12:58:48,650 [DEBUG]energy_py.envs.env_core: Episode 18 - Step 6
2017-12-20 12:58:48,650 [DEBUG]energy_py.envs.battery.battery_env: step is 6.000
2017-12-20 12:58:48,651 [DEBUG]energy_py.envs.battery.battery_env: action was [ 1.90376849  0.52620082]
2017-12-20 12:58:48,652 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.068 MWh
2017-12-20 12:58:48,652 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.171 MWh
2017-12-20 12:58:48,653 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 1.378 MW
2017-12-20 12:58:48,653 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.011 MWh
2017-12-20 12:58:48,654 [DEBUG]energy_py.envs.battery.battery_env: net rate is 1.240 MW
2017-12-20 12:58:48,655 [DEBUG]energy_py.envs.battery.battery_env: reward is -4.592 $/5min
2017-12-20 12:58:48,656 [DEBUG]energy_py.agents.memory: adding exp episode 18 step 6
2017-12-20 12:58:48,656 [DEBUG]energy_py.a

2017-12-20 12:58:48,753 [DEBUG]energy_py.agents.memory: adding exp episode 19 step 4
2017-12-20 12:58:48,754 [DEBUG]energy_py.agents.agent: Agent is acting
2017-12-20 12:58:48,755 [DEBUG]energy_py.envs.env_core: Episode 19 - Step 5
2017-12-20 12:58:48,755 [DEBUG]energy_py.envs.battery.battery_env: step is 5.000
2017-12-20 12:58:48,756 [DEBUG]energy_py.envs.battery.battery_env: action was [ 0.34705543  1.51589156]
2017-12-20 12:58:48,757 [DEBUG]energy_py.envs.battery.battery_env: old charge was 0.000 MWh
2017-12-20 12:58:48,757 [DEBUG]energy_py.envs.battery.battery_env: new charge is 0.000 MWh
2017-12-20 12:58:48,758 [DEBUG]energy_py.envs.battery.battery_env: gross rate is 0.000 MW
2017-12-20 12:58:48,759 [DEBUG]energy_py.envs.battery.battery_env: losses were 0.000 MWh
2017-12-20 12:58:48,760 [DEBUG]energy_py.envs.battery.battery_env: net rate is 0.000 MW
2017-12-20 12:58:48,761 [DEBUG]energy_py.envs.battery.battery_env: reward is -0.000 $/5min
2017-12-20 12:58:48,762 [DEBUG]energy_py.a

In [7]:
agent_outputs['df_ep']

,obs_0,obs_1,obs_2,obs_3,obs_4,obs_5,obs_6,obs_7,obs_8,obs_9,...,next_obs_5,next_obs_6,next_obs_7,next_obs_8,next_obs_9,next_obs_10,step,cum max reward,rolling mean,rolling std
episode,,,,,,,,,,,,,,,,,,,,,
1,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,0.457806,36,-5.319554,NaN,NaN
2,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,0.524460,36,-5.319554,-6.381337,1.501587
3,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,0.470914,36,-0.737968,-4.090544,4.741258
4,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,0.117323,36,-0.515768,-0.626868,0.157119
5,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,0.783369,36,-0.515768,-0.648719,0.188022
6,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,1.190891,36,-0.515768,-3.773620,4.231254
7,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,0.088962,36,-0.266061,-3.515814,4.595846
8,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,0.474999,36,-0.266061,-2.515950,3.181825
9,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,...,360.0,360.0,360.0,360.0,360.0,1.302014,36,-0.266061,-7.735502,4.199736
